<a href="https://colab.research.google.com/github/z4n0/LessonsExtractor/blob/main/LessonsExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
%ls /content/drive/MyDrive/lessonsExtractor

/content/drive/MyDrive/lessonsExtractor@


NB: TURN ON THE T4 GPU

In [18]:
!pip install opencv-python

In [40]:
%cd /content/drive/MyDrive/lessonsExtractor/

/content/drive/.shortcut-targets-by-id/1bQb8gi6zCwEeUgzwHQLPETmzfh9dMdVq/lessonsExtractor


In [41]:
%ls

9_Semantic_Segmentation_converted.mp4  converted_video.mp4  LessonsExtractor.ipynb  slides/
9_Semantic_Segmentation.mp4            CSR-20221124.mp4     output.docx


## conversion from mkv to mp4

In [ ]:
!ffmpeg -i prova.mkv

In [ ]:
#!ffmpeg -i prova.mkv -c:v libx264 -c:a aac prova_converted.mp4

reduce quality and bit rate to help the code run faster

In [ ]:
!ffmpeg -i prova.mkv -s 1280x720 -b:v 1M -c:v libx264 -c:a aac prova_converted.mp4

CODE TO REDUCE QUALITY AND BIT RATE AND SELECT A PIECE OF THE MP4 VIDEO

#check if ffmpeg is compatible for hw accelleration

TURN ON THE T4 GPU!!

A typical successful output snippet might start like this:


In [ ]:

Encoder h264_nvenc [NVIDIA NVENC H.264 encoder]:
    General capabilities: delay hardware
    Threading capabilities: none
    Supported pixel formats: nv12 p010le yuv444p nv16 yuv444p16le bgr0 rgb0 ...
    ...


In [ ]:
#code to check if you can use accelleration
!ffmpeg -h encoder=h264_nvenc

In [ ]:
#check if the tpu is on
!nvidia-smi

Thu Nov 16 17:06:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%ls

converted_video.mp4  CSR-20221124.mp4  lessonsExtractor.ipynb  slides/


#slide extractor

In [42]:
import cv2
import numpy as np
import os

In [43]:
input_video_path = '/content/drive/MyDrive/lessonsExtractor/9_Semantic_Segmentation.mp4'

# Extract base name and remove extension to form the new output file name
base_name = os.path.basename(input_video_path)
file_name_without_ext = os.path.splitext(base_name)[0]
output_video_path = f"/content/drive/MyDrive/lessonsExtractor/{file_name_without_ext}_converted.mp4"
resolution = "1280x720"
bitrate = "1M"
start_time = "00:40:00"  # Start at 40 minutes

In [ ]:
#duration = "00:30:00"  # Duration of 50 minutes, will stop at the end of the video if it's shorter

# Standard FFmpeg command
#!ffmpeg -ss {start_time} -i {input_video_path} -t {duration} -s {resolution} -b:v {bitrate} -c:v libx264 -c:a aac {output_video_path}

# GPU accelerated version
#!ffmpeg -hwaccel cuda -ss {start_time} -i {input_video_path} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}
!ffmpeg -i {input_video_path} -ss {start_time} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}

you should see the new converted_video.mp4 here. let's check!

In [ ]:
%ls

drive/  sample_data/


In [44]:
#TO BE DELETED?
def sum_time(time1, time2):
    # Convert time to seconds
    def time_to_seconds(time_str):
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s

    # Convert seconds back to time
    def seconds_to_time(seconds):
        h = seconds // 3600
        m = (seconds % 3600) // 60
        s = seconds % 60
        return f"{h:02d}:{m:02d}:{s:02d}"

    total_seconds = time_to_seconds(time1) + time_to_seconds(time2)
    return seconds_to_time(total_seconds)


**NB: THE PARAMETER THRESHOLD IN DETECT_SLIDES(....) HAS TO BE FINE TUNED FOR EACH VIDEO**

the higher the value the more selective it becomes

In [69]:
def save_frame(frame, timestamp, output_folder='slides'):
    print(sum_time(timestamp, start_time))
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    filename = f"{output_folder}/slide_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")

def frame_difference(prev_frame, curr_frame):
    # Compute the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, curr_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count

def milliseconds_to_hh_mm_ss(milliseconds):
    hours = int(milliseconds / 3600000)
    minutes = int((milliseconds % 3600000) / 60000)
    seconds = int((milliseconds % 60000) / 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

#trashold is to be fine tuned for each video
def detect_slides(video_path, threshold=385000, skip_frames=50):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            gray_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            if frame_difference(prev_frame, gray_frame) > threshold:
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                timestamp_readable = milliseconds_to_hh_mm_ss(timestamp_ms)
                adjusted_timestamp = sum_time(timestamp_readable, start_time)  # Adjust the timestamp
                slide_changes.append(adjusted_timestamp)
                save_frame(curr_frame, adjusted_timestamp)  # Use adjusted timestamp
                prev_frame = gray_frame
        frame_count += 1

    cap.release()
    return slide_changes

input_video_path = output_video_path
slide_changes = detect_slides(input_video_path)
print(f"Slide changes detected at: {slide_changes}")

01:20:26
Saved: slides/slide_00:40:26.jpg
01:20:56
Saved: slides/slide_00:40:56.jpg
01:21:18
Saved: slides/slide_00:41:18.jpg
01:21:32
Saved: slides/slide_00:41:32.jpg
01:22:52
Saved: slides/slide_00:42:52.jpg
01:26:00
Saved: slides/slide_00:46:00.jpg
01:28:12
Saved: slides/slide_00:48:12.jpg
01:28:18
Saved: slides/slide_00:48:18.jpg
01:28:20
Saved: slides/slide_00:48:20.jpg
01:28:40
Saved: slides/slide_00:48:40.jpg
01:28:54
Saved: slides/slide_00:48:54.jpg
01:29:10
Saved: slides/slide_00:49:10.jpg
01:29:40
Saved: slides/slide_00:49:40.jpg
01:29:56
Saved: slides/slide_00:49:56.jpg
01:30:00
Saved: slides/slide_00:50:00.jpg
01:30:20
Saved: slides/slide_00:50:20.jpg
01:30:32
Saved: slides/slide_00:50:32.jpg
01:30:40
Saved: slides/slide_00:50:40.jpg
01:30:46
Saved: slides/slide_00:50:46.jpg
01:30:48
Saved: slides/slide_00:50:48.jpg
01:34:06
Saved: slides/slide_00:54:06.jpg
01:35:20
Saved: slides/slide_00:55:20.jpg
01:38:38
Saved: slides/slide_00:58:38.jpg
01:39:12
Saved: slides/slide_00:59

In [70]:
print(f"detected slides {len(slide_changes)}")

detected slides 94


# Audio extraction

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-md7it0l5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-md7it0l5
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=49844c9dd639d33a3545d2d5709fb6161941f8816492d7809b18e01b583064b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ohld9ip/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [59]:
import whisper

def transcribe_video(video_path, model_size='base'):
    model = whisper.load_model(model_size)
    result = model.transcribe(video_path)
    return result

In [60]:
def interleave_transcription_with_slides(transcription, slide_changes):
    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes) and slide_changes[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

#pdf or docx creation

In [30]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.5 MB/s eta 0:00:00


In [71]:
from docx import Document
from docx.shared import Pt, Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.section import WD_SECTION_START


def create_document(interleaved_data, video_path, slides_folder='slides'):
    # Extract base name and remove extension to form the DOCX file name
    base_name = os.path.basename(video_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_file = f'{file_name_without_ext}.docx'

    doc = Document()

    style = doc.styles['Normal']
    style.paragraph_format.line_spacing = 1  # Single line spacing
    style.paragraph_format.space_before = Pt(0)
    style.paragraph_format.space_after = Pt(0)

    for item in interleaved_data:
        if item['type'] == 'slide_change':
      # Add slide image
            slide_image_path = f"{slides_folder}/slide_{item['timestamp']}.jpg"
            p = doc.add_paragraph()
            try:
                p.add_run().add_picture(slide_image_path, width=Inches(6))  # Adjust the width as needed
            except Exception as e:
                p.add_run(f"Could not load slide image: {slide_image_path}")
        else:
             # Clean up the transcribed text
            cleaned_text = item['text'].strip().replace('\n', ' ')
            # Add the cleaned text to the document
            p = doc.add_paragraph(cleaned_text)
            # Adjust paragraph formatting as needed
            p.paragraph_format.line_spacing = 1  # Single line spacing
            p.paragraph_format.space_before = Pt(0)
            p.paragraph_format.space_after = Pt(0)

    doc.save(output_file)
    print(f"Document saved as: {output_file}")


def convert_timestamp_to_seconds(timestamp_str):
    parts = timestamp_str.split(':')
    if len(parts) == 3:
        hours, minutes, seconds = map(int, parts)
        return hours * 3600 + minutes * 60 + seconds
    elif len(parts) == 2:
        minutes, seconds = map(int, parts)
        return minutes * 60 + seconds
    else:
        raise ValueError(f"Invalid timestamp format: {timestamp_str}")

def interleave_transcription_with_slides(transcription, slide_changes):
    # Convert slide change timestamps to seconds
    slide_changes_seconds = [convert_timestamp_to_seconds(ts) for ts in slide_changes]

    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes_seconds) and slide_changes_seconds[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes_seconds):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

# Transcribe video
transcription = transcribe_video(output_video_path)

# Interleave transcription with slide changes
interleaved_data = interleave_transcription_with_slides(transcription, slide_changes)

# Create DOCX document
create_document(interleaved_data, output_video_path)

Document saved as: 9_Semantic_Segmentation_converted.docx


In [38]:
%ls

9_Semantic_Segmentation_converted.mp4  converted_video.mp4  LessonsExtractor.ipynb  slides/
9_Semantic_Segmentation.mp4            CSR-20221124.mp4     output.docx


eliminate the slides images after having generated the docx output

In [68]:
%rm -rf /content/drive/MyDrive/lessonsExtractor/slides
